<a href="https://colab.research.google.com/github/mahdavipanah/IoT-Colab/blob/main/IoT_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link to the paper https://www.sciencedirect.com/science/article/pii/S0959652621016267

In [ ]:
pip install dataframe-image

     |████████████████████████████████| 1.1 MB 6.2 MB/s 
     |████████████████████████████████| 160 kB 58.5 MB/s 
     |████████████████████████████████| 271 kB 59.9 MB/s 
     |████████████████████████████████| 192 kB 50.8 MB/s 


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import dataframe_image as dfi

%load_ext google.colab.data_table

In [ ]:
#@title #Parameters

#@markdown Prior knowledge for the probability of the event occurrence:
PE = 0.6 #@param {type:"number"}

#@markdown Selection ratio in the list of nodes
S = 100 #@param {type:"integer"}
K = 10 #@param {type:"integer"}

#@markdown Number of trusted nodes:
TRUSTED_NODES_COUNT = 90 #@param {type:"integer"}

#@markdown Trusted nodes probability range (inclusive):
P_START = 0.5 #@param {type:"number"}
P_END = 0.9 #@param {type:"number"}


#@markdown Malicious nodes probability range (inclusive)
MALICIOUS_P_START = 0.3 #@param {type:"number"}
MALICIOUS_P_END = 0.39 #@param {type:"number"}

#@markdown Subtraction constants when calculating Bayesian inference for
#@markdown **cost2**:
SJ_SUBTRACT = 0.5 #@param {type:"number"}
PE_SUBTRACT = 0.5 #@param {type:"number"}


# trusted nodes
trusted_nodes = np.random.uniform(
  P_START,
  P_END,
  TRUSTED_NODES_COUNT
)
# malicious nodes 
malicious_nodes = np.random.uniform(
  MALICIOUS_P_START,
  MALICIOUS_P_END,
  S - TRUSTED_NODES_COUNT,
)


# concatinate the lists
nodes = np.concatenate([trusted_nodes, malicious_nodes])

data_frame = pd.DataFrame(nodes, columns = ['probability']) 

for n in np.arange(S//K):
  # indecies of the n-th E-Sensor
  sensor_indicies = np.arange(n * K, n * K + K)
  
  for i in sensor_indicies:
    i_prob = data_frame.at[i, 'probability']

    # indicies of other nodes of the sensor
    others_indicies = np.delete(
      sensor_indicies,
      np.where(sensor_indicies == i)
    )

    # average of other nodes of the sensor
    others_average = np.mean(
        data_frame.loc[others_indicies, 'probability'].values
    )
    data_frame.at[i, 'average'] = others_average

    # cost1
    i_cost1 = (i_prob - others_average) ** 2
    data_frame.at[i, 'cost1'] = i_cost1
    
    # cost2 calculations
    consistant_pi = 1
    non_consistent_p = 1
    
    for j in others_indicies:
      j_prob = data_frame.at[j, 'probability']

      condition = (j_prob - SJ_SUBTRACT) * (PE - PE_SUBTRACT)
      
      if condition >= 0:
        consistant_pi *= j_prob
      else:
        non_consistent_p *= j_prob
    
    pi_et = ((PE * consistant_pi) /
             (PE * consistant_pi + (1 - PE) * non_consistent_p))

    i_cost2 = (i_prob - pi_et) ** 2
    data_frame.at[i, 'cost2'] = i_cost2

    # utility
    data_frame.at[i, 'utility'] = (i_cost1 + i_cost2)

data_frame

,probability,average,cost1,cost2,utility
0,0.708556,0.670170,0.001474,0.452368,0.453841
1,0.690272,0.672201,0.000327,0.426906,0.427233
2,0.569000,0.685676,0.013613,0.275200,0.288813
3,0.649473,0.676734,0.000743,0.372533,0.373276
4,0.836585,0.655944,0.032631,0.649552,0.682183
...,...,...,...,...,...
95,0.375891,0.341328,0.001195,0.389462,0.390656
96,0.383728,0.340457,0.001872,0.379743,0.381615
97,0.359497,0.343149,0.000267,0.410190,0.410458
98,0.304234,0.349289,0.002030,0.484020,0.486050
